## Import Libs

In [1]:
import os
import sys
import pickle
import pandas as pd
import numpy as np
from numpy import sort
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, auc, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from plot_metric.functions import BinaryClassification
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from hyperopt import STATUS_OK, hp, fmin, tpe

In [3]:
# check the system font
import matplotlib.font_manager as font_manager
font_manager.findSystemFonts(fontpaths=None, fontext='ttf')

# add the font wanted
font_dir = ['../../Latin-Modern-Roman/']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

# Set font family globally
plt.rcParams['font.family'] = 'Latin Modern Roman'
print(plt.rcParams['font.family'])
plt.rcParams.update({'font.size': 14})

['Latin Modern Roman']


## Training Helper Functions

### Hyper-params Tuning

In [4]:
def BO_TPE(X_train, y_train, X_val, y_val):
    "Hyperparameter optimization"
    train = xgb.DMatrix(X_train, label=y_train)
    val = xgb.DMatrix(X_val, label=y_val)
    X_val_D = xgb.DMatrix(X_val)    #DMatrix is an internal data structure that is used by XGBoost, which is optimized for both memory efficiency and training speed.


    def objective(params):
        xgb_model = xgb.train(params, dtrain=train, num_boost_round=1000, evals=[(val, 'eval')],
                              verbose_eval=False, early_stopping_rounds=80)
        y_vd_pred = xgb_model.predict(X_val_D, ntree_limit=xgb_model.best_ntree_limit)
        y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]

        acc = accuracy_score(y_val, y_val_class)
        loss = 1 - acc

        return {'loss': loss, 'params': params, 'status': STATUS_OK}

    max_depths = [3, 4]
    learning_rates = [0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.15, 0.2]
    subsamples = [0.5, 0.6, 0.7, 0.8, 0.9]
    colsample_bytrees = [0.5, 0.6, 0.7, 0.8, 0.9]
    reg_alphas = [0.0, 0.005, 0.01, 0.05, 0.1]
    reg_lambdas = [0.8, 1, 1.5, 2, 4]

    space = {
        'max_depth': hp.choice('max_depth', max_depths),
        'learning_rate': hp.choice('learning_rate', learning_rates),
        'subsample': hp.choice('subsample', subsamples),
        'colsample_bytree': hp.choice('colsample_bytree', colsample_bytrees),
        'reg_alpha': hp.choice('reg_alpha', reg_alphas),
        'reg_lambda': hp.choice('reg_lambda', reg_lambdas),
    }

    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20)

    best_param = {'max_depth': max_depths[(best['max_depth'])],
                  'learning_rate': learning_rates[(best['learning_rate'])],
                  'subsample': subsamples[(best['subsample'])],
                  'colsample_bytree': colsample_bytrees[(best['colsample_bytree'])],
                  'reg_alpha': reg_alphas[(best['reg_alpha'])],
                  'reg_lambda': reg_lambdas[(best['reg_lambda'])]
                  }

    return best_param

### Train Model

In [5]:
def train_model(k, X_train, y_train, X_val, y_val, save_model_dir, fig_name_val, fig_name_fi, features):
    print('*************************************************************')
    print('{}th training ..............'.format(k + 1))
    print('Hyperparameters optimization')
    best_param = BO_TPE(X_train, y_train, X_val, y_val)
    print('Best Params: ', best_param)
    xgb_model = xgb.XGBClassifier(max_depth = best_param['max_depth'],
                                  eta = best_param['learning_rate'],
                                  n_estimators = 1000,
                                  subsample = best_param['subsample'],
                                  colsample_bytree = best_param['colsample_bytree'],
                                  reg_alpha = best_param['reg_alpha'],
                                  reg_lambda = best_param['reg_lambda'],
                                  objective = "binary:logistic"
                                  )

    xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='error',
                  early_stopping_rounds=80, verbose=False)

    # print training performance
    y_tr_pred = (xgb_model.predict_proba(X_train, ntree_limit=xgb_model.best_ntree_limit))[:, 1]
    train_auc = roc_auc_score(y_train, y_tr_pred)
    print('training dataset AUC: ' + str(train_auc))
    y_tr_class = [0 if i <= 0.5 else 1 for i in y_tr_pred]
    acc = accuracy_score(y_train, y_tr_class)
    print('training dataset acc: ' + str(acc))

    #print validation performance 
    y_vd_pred = (xgb_model.predict_proba(X_val, ntree_limit=xgb_model.best_ntree_limit))[:, 1]
    valid_auc = roc_auc_score(y_val, y_vd_pred)
    print('validation dataset AUROC: ' + str(valid_auc))
    precision_, recall_, thresholds_ = precision_recall_curve(y_val, y_vd_pred)
    auprc_ = auc(recall_, precision_)
    print('validation dataset AUPRC: ' + str(auprc_))
    y_val_class = [0 if i <= 0.5 else 1 for i in y_vd_pred]
    valid_acc = accuracy_score(y_val, y_val_class)
    print('validation dataset acc: ' + str(valid_acc))
    f1_ = f1_score(y_val,y_val_class)
    print('validation dataset F1: ' + str(f1_))
    fpr_, tpr_, thresholds_ = roc_curve(y_val, y_vd_pred)
    print('************************************************************')

    # save the model
    save_model_path = save_model_dir + 'model{}.mdl'.format(k + 1)
    xgb_model.get_booster().save_model(fname=save_model_path)

    # Plot ROC,PRC and confusion matrix
    bc = BinaryClassification(y_val,y_vd_pred, labels=['Non-sepsis', 'Sepsis'])
    # plt.figure(figsize=(11.69,8.27))
    plt.figure(figsize=(11.69,8.27))
    plt.subplot2grid(shape=(2,4), loc=(0,0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2,4), (0,2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2,4), (1,0), colspan=2)
    bc.plot_confusion_matrix()
    plt.subplot2grid((2,4), (1,2), colspan=2)
    bc.plot_confusion_matrix(normalize=True)
    plt.savefig('./figs_temp/{}_{}.pdf'.format(fig_name_val, k+1))
    plt.clf()

    # < ----- uncomment below for feature selection process

    # # Explore feature importance 
    # print('---Feature Importance---')
    # # plot feature importance
    # plot_importance(xgb_model)

    # #plot graph of feature importances for better visualization
    # feat_importances = pd.Series(xgb_model.feature_importances_, index=features)
    # feat_importances.nlargest(len(features)).plot(kind='barh')
    # plt.savefig('./figs/{}_{}.pdf'.format(fig_name_fi, k+1))
    # plt.clf()


    # # Fit model using each importance as a threshold 
    # thresholds = sort(xgb_model.feature_importances_)
    # # feature and performance 
    # for thresh in thresholds:
    #     # select features using threshold
    #     selection = SelectFromModel(xgb_model, threshold=thresh, prefit=True)
    #     select_X_train = selection.transform(X_train)
    #     # train model
    #     selection_model = XGBClassifier()
    #     selection_model.fit(select_X_train, y_train)
    #     # eval model
    #     select_X_test = selection.transform(X_val)
    #     predictions = selection_model.predict(select_X_test)
    #     accuracy = accuracy_score(y_val, predictions)
    #     print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

### Data Process

In [6]:
def gather_dfs(data_path, patient_ids):

    gathered_df = pd.DataFrame([])

    for p in tqdm(patient_ids):
        # read in patient data
        p = str(np.char.replace(p,'psv','csv'))
        df = pd.read_csv(data_path + p, sep = ",")
        if 'time' in df.columns: df = df.drop(['time'],axis=1)   # drop column time if exists
        gathered_df = pd.concat([gathered_df, df], ignore_index=True)
    
    return gathered_df

In [7]:
def expand_sepsis_label(df):

    number_nonsepsis_label = df.groupby('sepsis')['sepsis'].count()[0]
    print('Number of Non-sepsis label:', number_nonsepsis_label)
    number_sepsis_label = df.groupby('sepsis')['sepsis'].count()[1]
    print('Number of Sepsis label:', number_sepsis_label)

    expand_times = round((number_nonsepsis_label-number_sepsis_label)/number_sepsis_label)

    df_is_sepsis = df['sepsis']==1
    df_to_expand = df[df_is_sepsis]
    df_expanded = df.append([df_to_expand]*expand_times, ignore_index = True)

    number_nonsepsis_label = df_expanded.groupby('sepsis')['sepsis'].count()[0]
    print('After Expansion:\nNumber of Non-sepsis label:', number_nonsepsis_label)
    number_sepsis_label = df_expanded.groupby('sepsis')['sepsis'].count()[1]
    print('Number of Sepsis label:', number_sepsis_label)

    return df_expanded

## Train Model

In [13]:
!mkdir -p ./trained_models/BDWOFS/
!mkdir -p ./trained_models/BDWFS/
!mkdir -p ./trained_models/EDWOFS/
!mkdir -p ./trained_models/EDWFS/

### Baseline Data without Feature Selection

In [8]:
X_feature_baseline = ['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
                      'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
                      'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender']

y_feature = ['sepsis']

In [ ]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/baseline_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/baseline_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    # load the train and val data
    filename = '../data/data_both/kFold_baseline/fold{}/train.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        train_baseline = pickle.load(f)

    filename = '../data/data_both/kFold_baseline/fold{}/val.pickle'.format(k+1)
    with open(filename, 'rb') as f:
        val_baseline = pickle.load(f)

    x_train= train_baseline[X_feature_baseline]
    y_train = train_baseline[y_feature]

    x_val = val_baseline[X_feature_baseline]
    y_val = val_baseline[y_feature]


    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/BDWOFS/',
                fig_name_val = 'val_wo_fs_baseline',
                fig_name_fi = 'feature_importance_baseline',
                features = X_feature_baseline)   #change here

ACC:0.6575627855807575|F1:0.6580375529299552|AUROC:0.7168965795294563|AUPRC:0.6990500431431828

### Baseline Data with Feature Engineering

From the results from above, with the increase of the feature numbers , accuracy starts to decreases when only 10 most important features are used. As a results, the most important 10 features, along with other 5 vital signs will be selected as the final 15 features.

In [34]:
X_feature_baseline_fs = ['Magnesium', 'HCO3', 'MAP', 'Chloride', 'SaO2', 'Potassium', 'BaseExcess', 'Glucose', 'PH', 'gender',
                        'RR', 'SBP', 'Temp', 'DBP', 'HR']

y_feature = ['sepsis']

In [ ]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/baseline_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/baseline_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    # read data from Cinc2019
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    train_baseline_c = gather_dfs('../../datasets/Cinc2019/baseline_all/',train_set_c)
    val_baseline_c = gather_dfs('../../datasets/Cinc2019/baseline_all/',val_set_c)

    # read data from MIMIC-III 
    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    train_baseline_m = gather_dfs('../../datasets/MIMICIII/adults/baseline_all/',train_set_m)
    val_baseline_m = gather_dfs('../../datasets/MIMICIII/adults/baseline_all/',val_set_m)

    # combine two datasets
    train_baseline_both = pd.concat([train_baseline_c,train_baseline_m], ignore_index=True)
    train_baseline = expand_sepsis_label(train_baseline_both) #balance sepsis and non-sepsis labels
    val_baseline_both = pd.concat([val_baseline_c,val_baseline_m], ignore_index=True)
    val_baseline = expand_sepsis_label(val_baseline_both) #balance sepsis and non-sepsis labels

    x_train= train_baseline[X_feature_baseline_fs]
    y_train = train_baseline[y_feature]

    x_val = val_baseline[X_feature_baseline_fs]
    y_val = val_baseline[y_feature]


    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/BDWFS/',
                fig_name_val = 'val_w_fs_baseline',
                fig_name_fi = 'feature_importance_baseline',
                features = X_feature_baseline_fs)   #change here

ACC:0.6431193398244552|F1:0.639355669543864|AUROC:0.7024686209913262|AUPRC:0.6827815104252541

### Engineered Data without Feature Engineering

In [35]:
X_feature_engineered = ['HR', 'SaO2', 'Temp', 'SBP', 'MAP', 'DBP', 'RR', 'BaseExcess', 'HCO3',
       'PH', 'BUN', 'Calcium', 'Chloride', 'Creatinine', 'Glucose', 'Lactic',
       'Magnesium', 'Potassium', 'PTT', 'WBC', 'Platelet', 'age', 'gender',
       'HR_dev_1', 'HR_dev_2', 'HR_dev_3', 'RR_dev_1',
       'RR_dev_2', 'RR_dev_3', 'Temp_dev_1', 'Temp_dev_2', 'Temp_dev_3',
       'Bradycardia', 'Tachycardia', 'Hypothermia', 'Fever', 'Hyperpyrexia']

y_feature = ['sepsis']

In [ ]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/engineered_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/engineered_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    # read data from Cinc2019
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    train_engineered_c = gather_dfs('../../datasets/Cinc2019/engineered_all/',train_set_c)
    val_engineered_c = gather_dfs('../../datasets/Cinc2019/engineered_all/',val_set_c)

    # read data from MIMIC-III 
    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    train_engineered_m = gather_dfs('../../datasets/MIMICIII/adults/engineered_all/',train_set_m)
    val_engineered_m = gather_dfs('../../datasets/MIMICIII/adults/engineered_all/',val_set_m)

    # combine two datasets
    train_engineered_both = pd.concat([train_engineered_c,train_engineered_m], ignore_index=True)
    train_engineered = expand_sepsis_label(train_engineered_both) #balance sepsis and non-sepsis labels
    val_engineered_both = pd.concat([val_engineered_c,val_engineered_m], ignore_index=True)
    val_engineered = expand_sepsis_label(val_engineered_both) #balance sepsis and non-sepsis labels

    x_train= train_engineered[X_feature_engineered]
    y_train = train_engineered[y_feature]

    x_val = val_engineered[X_feature_engineered]
    y_val = val_engineered[y_feature]


    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/EDWOFS/',
                fig_name_val = 'val_wo_fs_engineered',
                fig_name_fi = 'feature_importance_engineered',
                features = X_feature_engineered)   #change here

ACC:0.8704827796512967|F1:0.8686430176531006|AUROC:0.9439847412700422|AUPRC:0.9359611161490434

### Engineered Data with Feature Engineering

When use 9-17 most important features, the model has higher accuracy. As a results, 17 most important features + 4 extra vital signs  = 21 features

In [36]:
X_feature_engineered_fs = ['RR_dev_3', 'Temp_dev_3', 'SaO2', 'Magnesium', 'MAP', 
                            'gender', 'Potassium', 'Chloride', 'PTT', 'SBP', 
                            'HCO3', 'BaseExcess', 'Glucose', 'WBC', 'BUN', 
                            'Calcium', 'Platelet', 
                            'RR', 'Temp', 'DBP', 'HR']

y_feature = ['sepsis']

In [ ]:
#load data from Cinc2019
data_path_c = "../../datasets/Cinc2019/engineered_all/"  
train_nonsepsis_c = np.load('../data/data_Cinc2019/train_nonsepsis.npy')
train_sepsis_c = np.load('../data/data_Cinc2019/train_sepsis.npy')

# load data from MIMIC-III
data_path_m = "../../datasets/MIMICIII/adults/engineered_all/"  
train_nonsepsis_m = np.load('../data/data_mimiciii/train_nonsepsis.npy')
train_sepsis_m = np.load('../data/data_mimiciii/train_sepsis.npy')

# 5-fold cross validation was implemented and five XGBoost models were produced
sss = StratifiedShuffleSplit(n_splits=5, test_size = 3/17, random_state=np.random.seed(12306)) # val/train = 0.15/0.7
for (k, (train0_index, val0_index)), (k, (train1_index, val1_index)), (k, (train2_index, val2_index)), (k, (train3_index, val3_index))\
     in zip(enumerate(sss.split(train_nonsepsis_c, np.zeros(train_nonsepsis_c.shape))), enumerate(sss.split(train_sepsis_c, np.zeros(train_sepsis_c.shape))), enumerate(sss.split(train_nonsepsis_m, np.zeros(train_nonsepsis_m.shape))), enumerate(sss.split(train_sepsis_m, np.zeros(train_sepsis_m.shape)))):

    print('---Fold{}/5---'.format(k+1))

    # read data from Cinc2019
    train_set_c = np.append((train_nonsepsis_c[train0_index])[:2052], (train_sepsis_c[train1_index])[:2052])
    val_set_c = np.append((train_nonsepsis_c[val0_index])[:440], (train_sepsis_c[val1_index][:440]))
    train_engineered_c = gather_dfs('../../datasets/Cinc2019/engineered_all/',train_set_c)
    val_engineered_c = gather_dfs('../../datasets/Cinc2019/engineered_all/',val_set_c)

    # read data from MIMIC-III 
    train_set_m = np.append((train_nonsepsis_m[train2_index][:1320]), (train_sepsis_m[train3_index])[:1320])
    val_set_m = np.append((train_nonsepsis_m[val2_index])[:283], (train_sepsis_m[val3_index])[:283])
    train_engineered_m = gather_dfs('../../datasets/MIMICIII/adults/engineered_all/',train_set_m)
    val_engineered_m = gather_dfs('../../datasets/MIMICIII/adults/engineered_all/',val_set_m)

    # combine two datasets
    train_engineered_both = pd.concat([train_engineered_c,train_engineered_m], ignore_index=True)
    train_engineered = expand_sepsis_label(train_engineered_both) #balance sepsis and non-sepsis labels
    val_engineered_both = pd.concat([val_engineered_c,val_engineered_m], ignore_index=True)
    val_engineered = expand_sepsis_label(val_engineered_both) #balance sepsis and non-sepsis labels

    x_train= train_engineered[X_feature_engineered_fs]
    y_train = train_engineered[y_feature]

    x_val = val_engineered[X_feature_engineered_fs]
    y_val = val_engineered[y_feature]


    print('---Start training process---')
    train_model(k, x_train, y_train, x_val, y_val, 
                save_model_dir = './trained_models/EDWFS/',
                fig_name_val = 'val_w_fs_engineered',
                fig_name_fi = 'feature_importance_engineered',
                features = X_feature_engineered_fs)   #change here

ACC:0.7901833103471321|F1:0.7826728469761315|AUROC:0.8679477638756825|AUPRC:0.8700686539486396

## Performance on Test data

### Score Function

In [10]:
#!/usr/bin/env python

# This file contains functions for evaluating algorithms for the 2019 PhysioNet/
# CinC Challenge. You can run it as follows:
#
#   python evaluate_sepsis_score.py labels predictions scores.csv
#
# where 'labels' is a directory containing files with labels, 'predictions' is a
# directory containing files with predictions, and 'scores.csv' (optional) is a
# collection of scores for the predictions.

################################################################################

# The evaluate_scores function computes a normalized utility score for a cohort
# of patients along with several traditional scoring metrics.
#
# Inputs:
#   'label_directory' is a directory of pipe-delimited text files containing a
#   binary vector of labels for whether a patient is not septic (0) or septic
#   (1) for each time interval.
#
#   'prediction_directory' is a directory of pipe-delimited text files, where
#   the first column of the file gives the predicted probability that the
#   patient is septic at each time, and the second column of the file is a
#   binarized version of this vector. Note that there must be a prediction for
#   every label.
#
# Outputs:
#   'auroc' is the area under the receiver operating characteristic curve
#   (AUROC).
#
#   'auprc' is the area under the precision recall curve (AUPRC).
#
#   'accuracy' is accuracy.
#
#   'f_measure' is F-measure.
#
#   'normalized_observed_utility' is a normalized utility-based measure that we
#   created for the Challenge. This score is normalized so that a perfect score
#   is 1 and no positive predictions is 0.
#
# Example:
#   Omitted due to length. See the below examples.

import numpy as np, os, os.path, sys, warnings

def evaluate_sepsis_score(label_directory, prediction_directory):
    # Set parameters.
    label_header       = 'sepsis'
    prediction_header  = 'PredictedLabel'
    probability_header = 'PredictedProbability'

    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3

    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0

    # Find label and prediction files.
    label_files = []
    for f in os.listdir(label_directory):
        g = os.path.join(label_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('csv'):
            label_files.append(g)
    label_files = sorted(label_files)

    prediction_files = []
    for f in os.listdir(prediction_directory):
        g = os.path.join(prediction_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('csv'):
            prediction_files.append(g)
    prediction_files = sorted(prediction_files)

    if len(label_files) != len(prediction_files):
        raise Exception('Numbers of label and prediction files must be the same.')

    # Load labels and predictions.
    num_files            = len(label_files)
    cohort_labels        = []
    cohort_predictions   = []
    cohort_probabilities = []

    for k in range(num_files):
        labels        = load_column(label_files[k], label_header, ',')
        predictions   = load_column(prediction_files[k], prediction_header, ',')
        probabilities = load_column(prediction_files[k], probability_header, ',')

        # Check labels and predictions for errors.
        if not (len(labels) == len(predictions) and len(predictions) == len(probabilities)):
            raise Exception('Numbers of labels and predictions for a file must be the same.')

        num_rows = len(labels)

        for i in range(num_rows):
            if labels[i] not in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

            if predictions[i] not in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

            if not 0 <= probabilities[i] <= 1:
                warnings.warn('Probabilities do not satisfy 0 <= probability <= 1.')

        if 0 < np.sum(predictions) < num_rows:
            min_probability_positive = np.min(probabilities[predictions == 1])
            max_probability_negative = np.max(probabilities[predictions == 0])

            if min_probability_positive <= max_probability_negative:
                warnings.warn('Predictions are inconsistent with probabilities, i.e., a positive prediction has a lower (or equal) probability than a negative prediction.')

        # Record labels and predictions.
        cohort_labels.append(labels)
        cohort_predictions.append(predictions)
        cohort_probabilities.append(probabilities)

    # Compute AUC, accuracy, and F-measure.
    labels        = np.concatenate(cohort_labels)
    predictions   = np.concatenate(cohort_predictions)
    probabilities = np.concatenate(cohort_probabilities)

    auroc, auprc        = compute_auc(labels, probabilities)
    accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

    # Compute utility.
    observed_utilities = np.zeros(num_files)
    best_utilities     = np.zeros(num_files)
    worst_utilities    = np.zeros(num_files)
    inaction_utilities = np.zeros(num_files)

    for k in range(num_files):
        labels = cohort_labels[k]
        num_rows          = len(labels)
        observed_predictions = cohort_predictions[k]
        best_predictions     = np.zeros(num_rows)
        worst_predictions    = np.zeros(num_rows)
        inaction_predictions = np.zeros(num_rows)

        if np.any(labels):
            t_sepsis = np.argmax(labels) - dt_optimal
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late + 1, num_rows)] = 1
        worst_predictions = 1 - best_predictions

        observed_utilities[k] = compute_prediction_utility(labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[k]     = compute_prediction_utility(labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[k]    = compute_prediction_utility(labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[k] = compute_prediction_utility(labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized_best_utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)

    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized_best_utility - unnormalized_inaction_utility)

    return auroc, auprc, accuracy, f_measure, normalized_observed_utility

# The load_column function loads a column from a table.
#
# Inputs:
#   'filename' is a string containing a filename.
#
#   'header' is a string containing a header.
#
# Outputs:
#   'column' is a vector containing a column from the file with the given
#   header.
#
# Example:
#   Omitted.

def load_column(filename, header, delimiter):
    column = []
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            arrs = l.strip().split(delimiter)
            if i == 0:
                try:
                    j = arrs.index(header)
                except:
                    raise Exception('{} must contain column with header {} containing numerical entries.'.format(filename, header))
            else:
                if len(arrs[j]):
                    column.append(float(arrs[j]))
    return np.array(column)

# The compute_auc function computes AUROC and AUPRC as well as other summary
# statistics (TP, FP, FN, TN, TPR, TNR, PPV, NPV, etc.) that can be exposed
# from this function.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a probability vector, where predictions[i] gives the
#   predicted probability that the patient is septic at time i.  Note that there
#   must be a prediction for every label, i.e, len(labels) ==
#   len(predictions).
#
# Outputs:
#   'auroc' is a scalar that gives the AUROC of the algorithm using its
#   predicted probabilities, where specificity is interpolated for intermediate
#   sensitivity values.
#
#   'auprc' is a scalar that gives the AUPRC of the algorithm using its
#   predicted probabilities, where precision is a piecewise constant function of
#   recall.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0.3, 0.4, 0.6, 0.7, 0.8, 0.8]
#   In [3]: auroc, auprc = compute_auc(labels, predictions)
#   In [4]: auroc
#   Out[4]: 1.0
#   In [5]: auprc
#   Out[5]: 1.0

def compute_auc(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not 0 <= prediction <= 1:
                warnings.warn('Predictions do not satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.insert(thresholds, 0, 1)
    if thresholds[-1] == 0:
        thresholds = thresholds[:-1]

    n = len(labels)
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort the predicted probabilities from largest to
    # smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels)
            tn[j] = n - fn[j]
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest predicted probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function with TPR /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant with TPR / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc

# The compute_accuracy_f_measure function computes the accuracy and F-measure
# for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'accuracy' is a scalar that gives the accuracy of the predictions using its
#   binarized predictions.
#
#   'f_measure' is a scalar that gives the F-measure of the predictions using its
#   binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)
#   In [4]: accuracy
#   Out[4]: 0.666666666667
#   In [5]: f_measure
#   Out[5]: 0.666666666667

def compute_accuracy_f_measure(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    n = len(labels)
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif not labels[i] and predictions[i]:
            fp += 1
        elif labels[i] and not predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

# The compute_prediction_utility function computes the total time-dependent
# utility for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'utility' is a scalar that gives the total time-dependent utility of the
#   algorithm using its binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: utility = compute_prediction_utility(labels, predictions)
#   In [4]: utility
#   Out[4]: 3.388888888888889

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

### Data Processing Functions

In [18]:
def save_challenge_predictions(file, scores, labels):
    with open(file, 'w') as f:
        f.write('PredictedProbability,PredictedLabel\n')
        for (s, l) in zip(scores, labels):
            f.write('%g,%d\n' % (s, l))

def save_challenge_testlabel(file, labels):
    with open(file, 'w') as f:
        f.write('sepsis\n')
        for l in labels:
            f.write('%d\n' % l)

def load_model_predict(X_test, k_fold, path):
    "ensemble the five XGBoost models by averaging their output probabilities"
    test_pred = np.zeros((X_test.shape[0], k_fold))
    X_test = xgb.DMatrix(X_test)
    for k in range(k_fold):
        # load the model
        model_path_name = path + 'model{}.mdl'.format(k+1) 
        loaded_model = xgb.Booster(model_file = model_path_name)
        # predict
        y_test_pred = loaded_model.predict(X_test)
        test_pred[:, k] = y_test_pred # save prediction results 5 times
    test_pred = pd.DataFrame(test_pred)
    result_pro = test_pred.mean(axis=1)

    return result_pro

def feature_extraction(case, data_features):
    labels = np.array(case['sepsis'])
    features = case[data_features]
    if 'time' in features.columns:
        features = features.drop(columns=['time'],axis = 1)

    return  features, labels    

def predict(data_set,
            data_dir,
            save_prediction_dir,
            save_label_dir,
            model_path,
            risk_threshold,
            data_features
            ):
    for csv in tqdm(data_set):
        csv = csv.replace('psv','csv')
        patient = pd.read_csv(data_dir+csv, sep=',')
        features, labels = feature_extraction(patient, data_features)

        predict_pro = load_model_predict(features, k_fold = 5, path = model_path)
        PredictedProbability = np.array(predict_pro)
        PredictedLabel = [0 if i <= risk_threshold else 1 for i in predict_pro]

        save_prediction_name = save_prediction_dir + csv
        save_challenge_predictions(save_prediction_name, PredictedProbability, PredictedLabel)
        save_testlabel_name = save_label_dir + csv
        save_challenge_testlabel(save_testlabel_name, labels)

### Baseline Data Without Feature Selection

In [12]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [13]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_baseline/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [00:29<00:00, 49.24it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.6430027131587179|0.1584866629878091|0.46124377795980087|0.18180774309609019|0.5792420020420319


### Baseline Data with Feature Selection

In [14]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [18]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_baseline/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [00:31<00:00, 46.19it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.6051262498204644|0.13278556543312744|0.3987524415600781|0.163622339866073|0.5454607334297628


### Engineered Data without Feature Selection

In [19]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [20]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_engineered/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [00:40<00:00, 35.63it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.9570821284661094|0.6842095605923605|0.8983806943481822|0.5730170496664195|0.793941440483281


### Engineered Data with Feature Selection

In [21]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [22]:
test_set = np.load('../data/data_both/test_set.npy')
test_data_path = '../data/data_both/test_engineered/' #change here
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' #change here

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs) # change feature here 

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 1446/1446 [00:36<00:00, 39.98it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.8891115972964225|0.5284534205776364|0.8372377291916073|0.42544483985765125|0.6983786054624351


## Test Real Neonatal Data

### BDWOFS

In [23]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [24]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:06<00:00, 32.91it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5335066020302109|0.041338095181773116|0.718327125277005|0.07979051139864449|0.13518218455274414


### BDWFS

In [25]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [26]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:06<00:00, 33.67it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5076501931378806|0.037213712441160736|0.05912584280258381|0.07198995489001535|0.2580747018987079


### EDWOFS

In [27]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [28]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:08<00:00, 27.20it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.9138986026434371|0.37454008785778076|0.8797680230091|0.323966065747614|0.6424446383314637


### EDWFS

In [29]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [30]:
# load test data
test_set = np.load('../../models_neoantes/data/test_set_balanced.npy')
test_data_path = '../../datasets/MIMICIII/neonates/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:07<00:00, 29.07it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.7864595194319878|0.25652378883274607|0.06110613418831628|0.07316732604142424|0.27419502264323403


## Test Artificial Neonatal Data
### BDWOFS

In [31]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [32]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:07<00:00, 31.09it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.7428517542833528|0.1048675334762386|0.8918151556686901|0.18570375829034635|0.2604512027919731


### BDWFS

In [33]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [34]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/baseline_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/BDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_baseline_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:06<00:00, 32.42it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.5577729790643288|0.04626758253852482|0.16291364793420796|0.07697290294720932|0.2885786696580665


### EDWSFS

In [35]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [36]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWOFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:08<00:00, 27.18it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.8988109213836714|0.44374816597385736|0.9505580575680439|0.45698924731182794|0.47739831318293247


### EDWFS

In [37]:
!rm -r -f ./prediction
!rm -r -f ./label
!mkdir -p ./prediction
!mkdir -p ./label

In [38]:
# load test data
test_set = np.load('../../artificial_neonatal_data/data/balanced_226/test_set_balanced.npy')
test_data_path = '../../artificial_neonatal_data/data/balanced_226/engineered_all/' 

# pathes
prediction_directory = './prediction/'
label_directory = './label/'
model_path = './trained_models/EDWFS/' 

predict(test_set, test_data_path, prediction_directory, label_directory, model_path, 0.5, X_feature_engineered_fs)

auroc, auprc, accuracy, f_measure, utility = evaluate_sepsis_score(label_directory, prediction_directory)
output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(
                auroc, auprc, accuracy, f_measure, utility)
print(output_string)

100%|██████████| 226/226 [00:05<00:00, 39.10it/s]


AUROC|AUPRC|Accuracy|F-measure|Utility
0.8147305967404469|0.34677461228986023|0.5512042294889368|0.12535775615340583|0.4800033237774731


## Generate Figs for test data

In [17]:
!mkdir -p ./figs/test/

In [31]:
def test_fig(data_set,
            data_dir,
            model_path,
            fig_name,
            risk_threshold,
            data_features):
    
    all_label = []
    all_predict = []

    for csv in tqdm(data_set):
        csv = csv.replace('psv','csv')
        patient = pd.read_csv(data_dir+csv, sep=',')
        features, labels = feature_extraction(patient, data_features)

        predict_pro = load_model_predict(features, k_fold = 5, path = model_path)
        PredictedProbability = np.array(predict_pro)
        PredictedLabel = [0 if i <= risk_threshold else 1 for i in predict_pro]

        all_label = np.append(all_label,labels)
        all_predict = np.append(all_predict,PredictedProbability)

    # Plot ROC,PRC and confusion matrix
    bc = BinaryClassification(all_label, all_predict, labels=['Non-sepsis', 'Sepsis'])
    plt.figure(figsize=(11.69,8.27))
    plt.subplot2grid(shape=(2,4), loc=(0,0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2,4), (0,2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2,4), (1,0), colspan=2)
    bc.plot_confusion_matrix()
    plt.subplot2grid((2,4), (1,2), colspan=2)
    bc.plot_confusion_matrix(normalize=True)
    plt.savefig('./figs/test/xgb_{}.pdf'.format(fig_name))
    plt.clf()


In [39]:
test_set = np.load('../data/data_both/test_set.npy')
#------------------change here-------------------------
test_data_path = '../data/data_both/test_engineered/' 
model_path = './trained_models/EDWFS/'
fig_name = 'EDWFS' 
X_feature = X_feature_engineered_fs
#------------------change here-------------------------

test_fig(test_set, test_data_path, model_path, fig_name, 0.5, X_feature) # change feature here 





100%|██████████| 1446/1446 [00:27<00:00, 51.95it/s]


79355
79355


<Figure size 841.68x595.44 with 0 Axes>